In [1]:
!git clone https://github.com/kumargopal/BgChanger.git

fatal: destination path 'BgChanger' already exists and is not an empty directory.


In [2]:
# prompt: mount gdrive

from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
%cd BgChanger

/content/BgChanger


In [4]:
!pip install "jax[cuda12_local]==0.4.23" -f https://storage.googleapis.com/jax-releases/jax_cuda_releases.html
!pip install diffusers==0.11.1
!pip install transformers scipy ftfy accelerate

Looking in links: https://storage.googleapis.com/jax-releases/jax_cuda_releases.html


In [5]:
!pip install flask-cloudflared

In [41]:
import os
import numpy as np
from flask import Flask, request, jsonify, render_template
from PIL import Image
import io
import base64
import time

# Flag to control local test mode
local_test_mode = True  # Change this to False for normal operation

# Initialize Flask app
app = Flask(__name__, template_folder='FrontEnd/templates',  static_folder='FrontEnd/static')


if not local_test_mode:
    from BgRemover.bg_remover import BgRemover
    from BgReplacer.StableDiffusionInPaint import StableDiffusionInPaint

    # Define paths and model files
    model_path = "/content/gdrive/My Drive"
    restore_model = "isnet.pth"
    background_remover = BgRemover(model_path, restore_model)

    inpaint_model_path = "runwayml/stable-diffusion-inpainting"
    inpaint_model = StableDiffusionInPaint(inpaint_model_path)

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/process', methods=['POST'])
def process():
    file = request.files['file']
    prompt = request.form['prompt']

    if local_test_mode:
        # Save the uploaded image locally
        image_path = 'uploaded_image.png'
    else:
        # Save the uploaded image to Google Drive
        image_path = '/content/gdrive/My Drive/uploaded_image.png'
    
    original_image = Image.open(file.stream)
    original_image.save(image_path)

    print("Image saved at:", image_path)
    print("Original image type:", type(original_image))

    if local_test_mode:
        # Simulate processing
        time.sleep(2)
        result_images = [original_image for _ in range(3)]
    else:
        # Load and process the image
        result_image = background_remover.remove_background(image_path)
        result_np = np.array(result_image)

        # Create mask image (white background, black foreground)
        mask_np = np.all(result_np[:, :, :3] == 0, axis=-1).astype(np.uint8) * 255
        mask_pil = Image.fromarray(mask_np)

        # Generate inpainted images
        result_images = inpaint_model.inpaint(result_image, mask_pil, prompt)

    # Encode result images to base64
    result_images_base64 = []
    for img in result_images:
        img_io = io.BytesIO()
        img.save(img_io, 'PNG')
        img_io.seek(0)
        img_base64 = base64.b64encode(img_io.read()).decode('utf-8')
        result_images_base64.append(f"data:image/png;base64,{img_base64}")

    return jsonify(result_images_base64)

from flask_cloudflared import run_with_cloudflared

run_with_cloudflared(app)
app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


 * Running on https://typical-equality-ipaq-monroe.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8686/metrics


127.0.0.1 - - [22/Jun/2024 15:44:25] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:29] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:32] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:32] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:32] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:33] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:33] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:35] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [22/Jun/2024 15:44:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:56] "GET /static/css/style.css HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:44:56] "GET /static/js/script.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Jun/2024 15:45:04] "GET /favicon.ico HTTP/1.1" 404 -


Image saved at: uploaded_image.png
Original image type: <class 'PIL.JpegImagePlugin.JpegImageFile'>


127.0.0.1 - - [22/Jun/2024 15:45:27] "POST /process HTTP/1.1" 200 -
